## Customer Segmentation with K-means

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

### Run k-means

In [4]:
datamart_rfm = pd.read_csv('datamart_rfm.csv')

In [7]:
datamart_log = np.log(datamart_rfm)


scaler = StandardScaler()
scaler.fit(datamart_log)


datamart_normalized = scaler.transform(datamart_log)

In [8]:

from sklearn.cluster import KMeans


kmeans = KMeans(n_clusters=3, random_state=1) 


kmeans.fit(datamart_normalized)


cluster_labels = kmeans.labels_

### Assign labels to raw data

In [ ]:

datamart_rfm_k3 = datamart_rfm.assign(Cluster=cluster_labels)


grouped = datamart_rfm_k3.groupby(['Cluster'])


grouped.agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
  }).round(1)